Tests for retrieving data from PACS
--

Imports
--

In [ ]:
import logging
import configparser
from datetime import datetime as dt

from IPython.core import display as ICD

from pydicom.dataset import Dataset

from scripts.run_all import run_all
from scripts.retrieve_data_from_PACS import retrieve_data

#from pynetdicom import debug_logger
#debug_logger()

%reload_ext autoreload
%autoreload 2

Set the logger of the pynetdicom module to ERROR, to avoid logs

In [ ]:
logging.getLogger('pynetdicom').setLevel(logging.ERROR)

Initialize the "config" object

In [ ]:
config = run_all()
#df_study = retrieve_studies(config)
#df_study

Get all the series corresponding to the query Dataset's filter [date = 2019-10-01, study time = between 8am and 9am]

In [ ]:
query_ds = Dataset()

# set the filtering parameters
query_ds.QueryRetrieveLevel = 'SERIES'
query_ds.StudyDate  = '20191001'
query_ds.SeriesDate = '20191001'
query_ds.StudyTime  = '080000-090000'
query_ds.SeriesTime = '080000-090000'

# parameters to fetch
query_ds.NumberOfSeriesRelatedInstances = ''
query_ds.SeriesInstanceUID = ''
query_ds.Modality = ''

logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

df_series_all = retrieve_data(config, query_ds)
df_series_all

Get a list of Series having more than a few frames

In [ ]:
df_series = df_series_all[df_series_all['Number of Series Related Instances'].astype(int) > 2][0:3]
df_series

Get time differences by querying the ContentTime on the IMAGE level

In [ ]:
time_format = '%H%M%S'

for i in df_series.index:
    
    series_UID = df_series.loc[i, 'Series Instance UID']
    last_frame = df_series.loc[i, 'Number of Series Related Instances']
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = ['1', last_frame]
    query_ds.SeriesInstanceUID = series_UID
    query_ds.ContentTime = ''
    
    df = retrieve_data(config, query_ds)
    
    start_time = dt.strptime(df.loc[0, 'Content Time'], time_format)
    end_time = dt.strptime(df.loc[1, 'Content Time'], time_format)
    delta_sec = (end_time - start_time).seconds

    logging.info('Series total time: end [{:%H:%M:%S}] - [{:%H:%M:%S}] = {} seconds'.format(end_time, start_time, delta_sec))

Get the Machine Name

In [ ]:
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.InstanceNumber = ['1', last_frame]
query_ds.SeriesInstanceUID = series_UID
query_ds.ContentTime = ''
query_ds.ManufacturerModelName = ''
    
logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)
    
df = retrieve_data(config, query_ds)
df

In [ ]:
df_series_pt = \
    df_series_all[
        (df_series_all.Modality == 'PT')
      & (df_series_all['Number of Series Related Instances'].astype(int) > 1)
    ][0:10]
df_series_pt

In [ ]:
time_format = '%H%M%S'

for i in df_series_pt.index:
    
    series_UID = df_series_pt.loc[i, 'Series Instance UID']
    last_frame = df_series_pt.loc[i, 'Number of Series Related Instances']
    mid_frame = str(int(int(last_frame) / 2))
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = ['1', mid_frame, last_frame]
    query_ds.SeriesInstanceUID = series_UID
    query_ds.StudyTime = ''
    query_ds.SeriesTime = ''
    query_ds.ContentTime = ''
    query_ds.NumberOfSeriesRelatedInstances = ''
    
    df = retrieve_data(config, query_ds)
    if len(df) > 0:
        ICD.display(df)

In [ ]:
df.loc[0, 'Content Time'].split('.')[0]